In [1]:
import numpy as np
import pandas as pd
from src.settings  import ROOT_DIR

data: pd.DataFrame = pd.read_pickle(ROOT_DIR / 'data' / 'raw' / 'anon_imu_data_time_series.pkl')
used_columns = ['anon_id', 'date_measure', 'time_stamp', 'imu_accX_right', 'imu_accX_left']
data = data[used_columns]
print(data.columns)

Index(['anon_id', 'date_measure', 'time_stamp', 'imu_accX_right',
       'imu_accX_left'],
      dtype='object')


In [2]:
data['combined_id_date'] = data['anon_id'].astype(str) + '_' + data['date_measure'].astype(str)
print(data.tail())

      anon_id       date_measure  time_stamp  imu_accX_right  imu_accX_left  \
1563      193  2023-8-4-11-52-39     32190.0        9.622112       9.355630   
1564      193  2023-8-4-11-52-39     32210.0        9.928983       9.257998   
1565      193  2023-8-4-11-52-39     32229.0       10.822164       9.209323   
1566      193  2023-8-4-11-52-39     32249.0       12.011940       9.158715   
1567      193  2023-8-4-11-52-39     32272.0       13.027627       9.061283   

           combined_id_date  
1563  193_2023-8-4-11-52-39  
1564  193_2023-8-4-11-52-39  
1565  193_2023-8-4-11-52-39  
1566  193_2023-8-4-11-52-39  
1567  193_2023-8-4-11-52-39  


In [3]:
df=data.copy()

In [6]:
# Suponiendo que tu DataFrame se llama df
# Asegúrate de tener numpy y pandas instalados: pip install numpy pandas

# Agrupa los datos por 'date_measure'
grouped_data = df.groupby('combined_id_date')

# Inicializa listas para almacenar los resultados
energy_data = []

# Itera sobre cada grupo
for combined_id_date, group in grouped_data:
    # Obtén las señales de aceleración de ambos lados
    accX_right = group['imu_accX_right'].values
    accX_left = group['imu_accX_left'].values

    # Calcula la FFT de cada señal de aceleración
    fft_result_right = np.fft.fft(accX_right)
    fft_result_left = np.fft.fft(accX_left)

    # Calcula la magnitud de la FFT (utiliza np.abs para obtener el valor absoluto)
    fft_magnitude_right = np.abs(fft_result_right)
    fft_magnitude_left = np.abs(fft_result_left)

    # Calcula la energía total para cada acelerómetro
    total_energy_right = np.sum(fft_magnitude_right)
    total_energy_left = np.sum(fft_magnitude_left)

    # Almacena la energía total en la lista
    energy_data.append({'combined_id_date': combined_id_date, 'total_energy_right': total_energy_right, 'total_energy_left': total_energy_left})

# Convierte la lista de resultados en un DataFrame
energy_df = pd.DataFrame(energy_data)

# Imprime el DataFrame con la energía total por medida
print(energy_df.columns)

Index(['combined_id_date', 'total_energy_right', 'total_energy_left'], dtype='object')


In [7]:
# Suponiendo que 'energy_df' es tu DataFrame con los resultados de energía
specific_combined_id_date = '10_2023-6-22-10-28-8'

# Filtra el DataFrame para obtener solo las filas correspondientes a la fecha específica
specific_specific_combined_id_date_data = energy_df.loc[energy_df['combined_id_date'] == specific_combined_id_date]

# Muestra los resultados
print(specific_specific_combined_id_date_data)


        combined_id_date  total_energy_right  total_energy_left
12  10_2023-6-22-10-28-8        85653.639433      115495.635742
